In [1]:
import numpy as np 
import pandas as pd 
import re
import os
import glob
import pickle
from sklearn.model_selection import train_test_split 
import librosa
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras import optimizers
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
INPUT_FOLDER = "input/"
# INPUT_FOLDER = "../input/"
print(os.listdir(INPUT_FOLDER))

['train_curated.csv', 'train_noisy.csv', 'sample_submission.csv', 'train_noisy', 'train_curated', 'test']


In [4]:
TRAIN_CURATED_PATH = INPUT_FOLDER + "train_curated.csv"
TRAIN_NOISY_PATH = INPUT_FOLDER + "train_noisy.csv"
SAMPLE_SUBMISSION_PATH = INPUT_FOLDER + "sample_submission.csv"
TRAIN_CURATED = INPUT_FOLDER + "train_curated/"
TRAIN_NOISY = INPUT_FOLDER + "train_noisy/"
TEST = INPUT_FOLDER + "test/"

train_curated = pd.read_csv(TRAIN_CURATED_PATH)
train_noisy = pd.read_csv(TRAIN_NOISY_PATH)
sample = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [5]:
def one_hot(labels, src_dict):
    ar = np.zeros([len(labels), len(src_dict)])
    for i, label in enumerate(labels):
        label_list = label.split(',')
        for la in label_list:
            ar[i, src_dict[la]] = 1
    return ar

In [6]:
target_names = sample.columns[1:]
num_targets = len(target_names)

src_dict = {target_names[i]:i for i in range(num_targets)}
src_dict_inv = {i:target_names[i] for i in range(num_targets)}

In [7]:
num_freq = 128
len_div = 256

In [8]:
inputs = Input(shape=(num_freq,len_div,1), name='input')

dense_list = []

## Block 1
conv1 = Conv2D(4, (19, 19),activation='relu',padding='same',name='conv1')(inputs)
pool1 = MaxPooling2D((19, 19),strides=(1, 1),padding='same',name='pool1')(conv1)
norm1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1')(pool1)
drop1 = Dropout(rate=0.05)(norm1)

conv1_1 = Conv2D(4, (11, 11),activation='relu',padding='same',name='conv1_1')(drop1)
pool1_1 = MaxPooling2D((5, 5),strides=(5, 5),padding='same',name='pool1_1')(conv1_1)
norm1_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_1')(pool1_1)
drop1_1 = Dropout(rate=0.05)(norm1_1)

flatten1 = Flatten(name='flatten1')(drop1)
dense1 = Dense(16, name='dense1')(flatten1)
act1 = Activation('relu',name='act1')(dense1)
dense_list.append(act1)

## Block 2
conv2 = Conv2D(4, (13, 13),activation='relu',padding='same',name='conv2')(inputs)
pool2 = MaxPooling2D((13, 13), strides=(1, 1), padding='same',name='pool2')(conv2)
norm2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2')(pool2)
drop2 = Dropout(rate=0.05)(norm2)

conv2_1 = Conv2D(4, (7, 7),activation='relu',padding='same',name='conv2_1')(drop2)
pool2_1 = MaxPooling2D((7, 7), strides=(5, 5), padding='same',name='pool2_1')(conv2_1)
norm2_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_1')(pool2_1)
drop2_1 = Dropout(rate=0.05)(norm2_1)

flatten2 = Flatten(name='flatten2')(drop2_1)
dense2 = Dense(16, name='dense2')(flatten2)
act2 = Activation('relu',name='act2')(dense2)
dense_list.append(act2)

## Block 3
conv3 = Conv2D(8, (11, 11), activation='relu',padding='same',name='conv3')(inputs)
pool3 = MaxPooling2D((11, 11), strides=(2, 2), padding='same',name='pool3')(conv3)
norm3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3')(pool3)
drop3 = Dropout(rate=0.05)(norm3)

conv3_1 = Conv2D(8, (5, 5), activation='relu',padding='same',name='conv3_1')(drop3)
pool3_1 = MaxPooling2D((5, 5), strides=(2, 2), padding='same',name='pool3_1')(conv3_1)
norm3_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3_1')(pool3_1)
drop3_1 = Dropout(rate=0.05)(norm3_1)

flatten3 = Flatten(name='flatten3')(drop3_1)
dense3 = Dense(16, name='dense3')(flatten3)
act3 = Activation('relu',name='act3')(dense3)
dense_list.append(act3)

## Block 4
conv4 = Conv2D(8, (9, 9),activation='relu',padding='same',name='conv4')(inputs)
pool4 = MaxPooling2D((9, 9), strides=(2, 2), padding='same',name='pool4')(conv4)
norm4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4')(pool4)
drop4 = Dropout(rate=0.05)(norm4)

conv4_1 = Conv2D(8, (3, 3),activation='relu',padding='same',name='conv4_1')(drop4)
pool4_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same',name='pool4_1')(conv4_1)
norm4_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4_1')(pool4_1)
drop4_1 = Dropout(rate=0.05)(norm4_1)

flatten4 = Flatten(name='flatten4')(drop4_1)
dense4 = Dense(16, name='dense4')(flatten4)
act4 = Activation('relu',name='act4')(dense4)
dense_list.append(act4)

concat = concatenate(dense_list, name='concat', axis=1)

dense2 = Dense(80, name='dense_all')(concat)
pred = Activation('softmax',name='pred')(dense2)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model = Model(inputs=inputs, outputs=pred)

In [9]:
def tf_one_sample_positive_class_precisions(y_true, y_pred) :
    num_samples, num_classes = y_pred.shape
    
    # find true labels
    pos_class_indices = tf.where(y_true > 0) 
    
    # put rank on each element
    retrieved_classes = tf.nn.top_k(y_pred, k=num_classes).indices
    sample_range = tf.zeros(shape=tf.shape(tf.transpose(y_pred)), dtype=tf.int32)
    sample_range = tf.add(sample_range, tf.range(tf.shape(y_pred)[0], delta=1))
    sample_range = tf.transpose(sample_range)
    sample_range = tf.reshape(sample_range, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_classes = tf.reshape(retrieved_classes, (-1,num_classes*tf.shape(y_pred)[0]))
    retrieved_class_map = tf.concat((sample_range, retrieved_classes), axis=0)
    retrieved_class_map = tf.transpose(retrieved_class_map)
    retrieved_class_map = tf.reshape(retrieved_class_map, (tf.shape(y_pred)[0], num_classes, 2))
    
    class_range = tf.zeros(shape=tf.shape(y_pred), dtype=tf.int32)
    class_range = tf.add(class_range, tf.range(num_classes, delta=1))
    
    class_rankings = tf.scatter_nd(retrieved_class_map,
                                          class_range,
                                          tf.shape(y_pred))
    
    #pick_up ranks
    num_correct_until_correct = tf.gather_nd(class_rankings, pos_class_indices)

    # add one for division for "presicion_at_hits"
    num_correct_until_correct_one = tf.add(num_correct_until_correct, 1) 
    num_correct_until_correct_one = tf.cast(num_correct_until_correct_one, tf.float32)
    
    # generate tensor [num_sample, predict_rank], 
    # top-N predicted elements have flag, N is the number of positive for each sample.
    sample_label = pos_class_indices[:, 0]   
    sample_label = tf.reshape(sample_label, (-1, 1))
    sample_label = tf.cast(sample_label, tf.int32)
    
    num_correct_until_correct = tf.reshape(num_correct_until_correct, (-1, 1))
    retrieved_class_true_position = tf.concat((sample_label, 
                                               num_correct_until_correct), axis=1)
    retrieved_pos = tf.ones(shape=tf.shape(retrieved_class_true_position)[0], dtype=tf.int32)
    retrieved_class_true = tf.scatter_nd(retrieved_class_true_position, 
                                         retrieved_pos, 
                                         tf.shape(y_pred))
    # cumulate predict_rank
    retrieved_cumulative_hits = tf.cumsum(retrieved_class_true, axis=1)

    # find positive position
    pos_ret_indices = tf.where(retrieved_class_true > 0)

    # find cumulative hits
    correct_rank = tf.gather_nd(retrieved_cumulative_hits, pos_ret_indices)  
    correct_rank = tf.cast(correct_rank, tf.float32)

    # compute presicion
    precision_at_hits = tf.truediv(correct_rank, num_correct_until_correct_one)

    return pos_class_indices, precision_at_hits

def tf_lwlrap(y_true, y_pred):
    num_samples, num_classes = y_pred.shape
    pos_class_indices, precision_at_hits = (tf_one_sample_positive_class_precisions(y_true, y_pred))
    pos_flgs = tf.cast(y_true > 0, tf.int32)
    labels_per_class = tf.reduce_sum(pos_flgs, axis=0)
    weight_per_class = tf.truediv(tf.cast(labels_per_class, tf.float32),
                                  tf.cast(tf.reduce_sum(labels_per_class), tf.float32))
    sum_precisions_by_classes = tf.zeros(shape=(num_classes), dtype=tf.float32)  
    class_label = pos_class_indices[:,1]
    sum_precisions_by_classes = tf.unsorted_segment_sum(precision_at_hits,
                                                        class_label,
                                                       num_classes)
    labels_per_class = tf.cast(labels_per_class, tf.float32)
    labels_per_class = tf.add(labels_per_class, 1e-7)
    per_class_lwlrap = tf.truediv(sum_precisions_by_classes,
                                  tf.cast(labels_per_class, tf.float32))
    out = tf.cast(tf.tensordot(per_class_lwlrap, weight_per_class, axes=1), dtype=tf.float32)
    return out

In [10]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=[tf_lwlrap])

In [11]:
with open('train_arr.pickle', 'rb') as f:
    X = pickle.load(f)
    y = pickle.load(f)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

batch_size = 64
epochs = 30

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val))

Train on 7501 samples, validate on 1876 samples
Epoch 1/30
7501/7501 [==============================] - 100s 13ms/step - loss: 6.6894 - tf_lwlrap: 0.1405 - val_loss: 5.2215 - val_tf_lwlrap: 0.1645
Epoch 2/30
7501/7501 [==============================] - 96s 13ms/step - loss: 4.6018 - tf_lwlrap: 0.2416 - val_loss: 4.5076 - val_tf_lwlrap: 0.2637
Epoch 3/30
7501/7501 [==============================] - 96s 13ms/step - loss: 4.0448 - tf_lwlrap: 0.3339 - val_loss: 4.3719 - val_tf_lwlrap: 0.3197
Epoch 4/30
7501/7501 [==============================] - 95s 13ms/step - loss: 3.7677 - tf_lwlrap: 0.4045 - val_loss: 4.0725 - val_tf_lwlrap: 0.3472
Epoch 5/30
7501/7501 [==============================] - 95s 13ms/step - loss: 3.4215 - tf_lwlrap: 0.4636 - val_loss: 4.1820 - val_tf_lwlrap: 0.3801
Epoch 6/30
7501/7501 [==============================] - 95s 13ms/step - loss: 3.1062 - tf_lwlrap: 0.5178 - val_loss: 4.0289 - val_tf_lwlrap: 0.3935
Epoch 7/30
7501/7501 [==============================] - 95s 13m

In [12]:
file_name = train_noisy['fname'].values

div = 500
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))

y_proc_tmp = one_hot(train_noisy['labels'], src_dict)

for e in range(epochs):
    k_list = random.sample(range(num_batch+1), num_batch+1)
    for k in k_list:
        with open('out/train_noisy_arr_{}.pickle'.format(k), 'rb') as f:
            X = pickle.load(f)
            y = pickle.load(f)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0) 
        model.fit(X_train, y_train, batch_size=batch_size,epochs=3,validation_data=(X_val, y_val))
        
print('noisy iter epoch No.{} is done.'.format(e))

Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 16s 14ms/step - loss: 8.3870 - tf_lwlrap: 0.1482 - val_loss: 5.7569 - val_tf_lwlrap: 0.1125
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 5.3571 - tf_lwlrap: 0.1287 - val_loss: 5.8636 - val_tf_lwlrap: 0.1153
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 5.2384 - tf_lwlrap: 0.1614 - val_loss: 5.7847 - val_tf_lwlrap: 0.1331
Train on 1183 samples, validate on 296 samples
Epoch 1/3
1183/1183 [==============================] - 16s 14ms/step - loss: 5.2986 - tf_lwlrap: 0.1240 - val_loss: 5.3884 - val_tf_lwlrap: 0.1235
Epoch 2/3
1183/1183 [==============================] - 15s 13ms/step - loss: 5.1220 - tf_lwlrap: 0.1414 - val_loss: 5.3204 - val_tf_lwlrap: 0.1382
Epoch 3/3
1183/1183 [==============================] - 15s 13ms/step - loss: 4.9943 - tf_lwlrap: 0.1622 - val_loss: 5.3313 - val_tf_lwlrap: 0.1639
Train on 1178 samples, v

1170/1170 [==============================] - 16s 14ms/step - loss: 4.2729 - tf_lwlrap: 0.3007 - val_loss: 4.3981 - val_tf_lwlrap: 0.2592
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 16s 13ms/step - loss: 4.6961 - tf_lwlrap: 0.2353 - val_loss: 4.7401 - val_tf_lwlrap: 0.2813
Epoch 2/3
1176/1176 [==============================] - 16s 13ms/step - loss: 4.5285 - tf_lwlrap: 0.2765 - val_loss: 4.6836 - val_tf_lwlrap: 0.2772
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 4.3489 - tf_lwlrap: 0.3038 - val_loss: 4.5622 - val_tf_lwlrap: 0.2880
Train on 1173 samples, validate on 294 samples
Epoch 1/3
1173/1173 [==============================] - 16s 13ms/step - loss: 4.8990 - tf_lwlrap: 0.2511 - val_loss: 4.6354 - val_tf_lwlrap: 0.2622
Epoch 2/3
1173/1173 [==============================] - 15s 13ms/step - loss: 4.6259 - tf_lwlrap: 0.2786 - val_loss: 4.5666 - val_tf_lwlrap: 0.2725
Epoch 3/3
1173/1173 [=============

1176/1176 [==============================] - 15s 13ms/step - loss: 3.8218 - tf_lwlrap: 0.3533 - val_loss: 4.0449 - val_tf_lwlrap: 0.3450
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.6252 - tf_lwlrap: 0.3900 - val_loss: 3.9971 - val_tf_lwlrap: 0.3460
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 15s 13ms/step - loss: 4.5400 - tf_lwlrap: 0.3095 - val_loss: 4.3796 - val_tf_lwlrap: 0.2958
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 4.2788 - tf_lwlrap: 0.3551 - val_loss: 4.2902 - val_tf_lwlrap: 0.3440
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 4.0884 - tf_lwlrap: 0.3994 - val_loss: 4.2663 - val_tf_lwlrap: 0.3372
Train on 1168 samples, validate on 293 samples
Epoch 1/3
1168/1168 [==============================] - 15s 13ms/step - loss: 4.3321 - tf_lwlrap: 0.2921 - val_loss: 4.2844 - val_tf_lwlrap: 0.3075
Epoch 2/3
1168/1168 [=============

1168/1168 [==============================] - 15s 13ms/step - loss: 4.0364 - tf_lwlrap: 0.3176 - val_loss: 4.0466 - val_tf_lwlrap: 0.3785
Epoch 2/3
1168/1168 [==============================] - 15s 13ms/step - loss: 3.7257 - tf_lwlrap: 0.3822 - val_loss: 4.0204 - val_tf_lwlrap: 0.3863
Epoch 3/3
1168/1168 [==============================] - 15s 13ms/step - loss: 3.5668 - tf_lwlrap: 0.4147 - val_loss: 3.9181 - val_tf_lwlrap: 0.3887
Train on 1169 samples, validate on 293 samples
Epoch 1/3
1169/1169 [==============================] - 15s 13ms/step - loss: 4.1408 - tf_lwlrap: 0.3406 - val_loss: 4.0388 - val_tf_lwlrap: 0.3596
Epoch 2/3
1169/1169 [==============================] - 15s 13ms/step - loss: 3.8367 - tf_lwlrap: 0.3863 - val_loss: 3.9051 - val_tf_lwlrap: 0.3804
Epoch 3/3
1169/1169 [==============================] - 15s 13ms/step - loss: 3.6582 - tf_lwlrap: 0.4280 - val_loss: 3.8832 - val_tf_lwlrap: 0.3880
Train on 1168 samples, validate on 293 samples
Epoch 1/3
1168/1168 [=============

Train on 1175 samples, validate on 294 samples
Epoch 1/3
1175/1175 [==============================] - 15s 13ms/step - loss: 4.0585 - tf_lwlrap: 0.3626 - val_loss: 4.3606 - val_tf_lwlrap: 0.3615
Epoch 2/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.8127 - tf_lwlrap: 0.4239 - val_loss: 4.0766 - val_tf_lwlrap: 0.4194
Epoch 3/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.6239 - tf_lwlrap: 0.4582 - val_loss: 4.0470 - val_tf_lwlrap: 0.4073
Train on 1183 samples, validate on 296 samples
Epoch 1/3
1183/1183 [==============================] - 15s 13ms/step - loss: 4.1646 - tf_lwlrap: 0.3500 - val_loss: 4.4360 - val_tf_lwlrap: 0.3209
Epoch 2/3
1183/1183 [==============================] - 15s 13ms/step - loss: 3.8587 - tf_lwlrap: 0.4029 - val_loss: 4.3680 - val_tf_lwlrap: 0.3372
Epoch 3/3
1183/1183 [==============================] - 15s 13ms/step - loss: 3.6464 - tf_lwlrap: 0.4490 - val_loss: 4.2929 - val_tf_lwlrap: 0.3538
Train on 1170 samples, v

...

1175/1175 [==============================] - 15s 13ms/step - loss: 2.9368 - tf_lwlrap: 0.5466 - val_loss: 4.2859 - val_tf_lwlrap: 0.4008
Epoch 2/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.5743 - tf_lwlrap: 0.6206 - val_loss: 4.1776 - val_tf_lwlrap: 0.4147
Epoch 3/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.3938 - tf_lwlrap: 0.6561 - val_loss: 4.2463 - val_tf_lwlrap: 0.4234
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.2869 - tf_lwlrap: 0.5326 - val_loss: 4.7656 - val_tf_lwlrap: 0.3450
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.9109 - tf_lwlrap: 0.6076 - val_loss: 4.7287 - val_tf_lwlrap: 0.3426
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.6731 - tf_lwlrap: 0.6422 - val_loss: 4.5776 - val_tf_lwlrap: 0.3670
Train on 1167 samples, validate on 292 samples
Epoch 1/3
1167/1167 [=============

Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.2131 - tf_lwlrap: 0.5515 - val_loss: 4.2030 - val_tf_lwlrap: 0.4483
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.9820 - tf_lwlrap: 0.5917 - val_loss: 4.3244 - val_tf_lwlrap: 0.4380
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.7536 - tf_lwlrap: 0.6429 - val_loss: 4.3037 - val_tf_lwlrap: 0.4560
Train on 1172 samples, validate on 293 samples
Epoch 1/3
1172/1172 [==============================] - 15s 13ms/step - loss: 3.1094 - tf_lwlrap: 0.5511 - val_loss: 4.1061 - val_tf_lwlrap: 0.4263
Epoch 2/3
1172/1172 [==============================] - 15s 13ms/step - loss: 2.7759 - tf_lwlrap: 0.6186 - val_loss: 4.0921 - val_tf_lwlrap: 0.4200
Epoch 3/3
1172/1172 [==============================] - 15s 13ms/step - loss: 2.5737 - tf_lwlrap: 0.6745 - val_loss: 4.1485 - val_tf_lwlrap: 0.4099
Train on 1174 samples, v

1171/1171 [==============================] - 15s 13ms/step - loss: 2.7763 - tf_lwlrap: 0.6233 - val_loss: 4.2752 - val_tf_lwlrap: 0.3968
Train on 1170 samples, validate on 293 samples
Epoch 1/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.2925 - tf_lwlrap: 0.5287 - val_loss: 4.0980 - val_tf_lwlrap: 0.4135
Epoch 2/3
1170/1170 [==============================] - 15s 13ms/step - loss: 2.9439 - tf_lwlrap: 0.5913 - val_loss: 3.9348 - val_tf_lwlrap: 0.4488
Epoch 3/3
1170/1170 [==============================] - 15s 13ms/step - loss: 2.7219 - tf_lwlrap: 0.6425 - val_loss: 3.9678 - val_tf_lwlrap: 0.4467
Train on 736 samples, validate on 185 samples
Epoch 1/3
736/736 [==============================] - 9s 13ms/step - loss: 3.2256 - tf_lwlrap: 0.5079 - val_loss: 3.8359 - val_tf_lwlrap: 0.4089
Epoch 2/3
736/736 [==============================] - 9s 13ms/step - loss: 2.8597 - tf_lwlrap: 0.5828 - val_loss: 3.7433 - val_tf_lwlrap: 0.4001
Epoch 3/3
736/736 [======================

1168/1168 [==============================] - 15s 13ms/step - loss: 2.7964 - tf_lwlrap: 0.5732 - val_loss: 4.0918 - val_tf_lwlrap: 0.4163
Epoch 3/3
1168/1168 [==============================] - 15s 13ms/step - loss: 2.6113 - tf_lwlrap: 0.6159 - val_loss: 4.1659 - val_tf_lwlrap: 0.4179
Train on 1184 samples, validate on 297 samples
Epoch 1/3
1184/1184 [==============================] - 15s 13ms/step - loss: 3.2712 - tf_lwlrap: 0.5361 - val_loss: 4.3506 - val_tf_lwlrap: 0.3426
Epoch 2/3
1184/1184 [==============================] - 15s 13ms/step - loss: 3.0446 - tf_lwlrap: 0.5803 - val_loss: 4.2803 - val_tf_lwlrap: 0.3709
Epoch 3/3
1184/1184 [==============================] - 15s 13ms/step - loss: 2.8045 - tf_lwlrap: 0.6347 - val_loss: 4.2099 - val_tf_lwlrap: 0.3756
Train on 1180 samples, validate on 295 samples
Epoch 1/3
1180/1180 [==============================] - 15s 13ms/step - loss: 3.2028 - tf_lwlrap: 0.5852 - val_loss: 4.2199 - val_tf_lwlrap: 0.4325
Epoch 2/3
1180/1180 [=============

1176/1176 [==============================] - 15s 13ms/step - loss: 3.3777 - tf_lwlrap: 0.5332 - val_loss: 4.4986 - val_tf_lwlrap: 0.3931
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.0723 - tf_lwlrap: 0.5895 - val_loss: 4.0472 - val_tf_lwlrap: 0.4399
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.8738 - tf_lwlrap: 0.6395 - val_loss: 4.2800 - val_tf_lwlrap: 0.4423
Train on 1171 samples, validate on 293 samples
Epoch 1/3
1171/1171 [==============================] - 15s 13ms/step - loss: 3.3677 - tf_lwlrap: 0.5256 - val_loss: 4.9347 - val_tf_lwlrap: 0.4153
Epoch 2/3
1171/1171 [==============================] - 15s 13ms/step - loss: 3.0545 - tf_lwlrap: 0.5868 - val_loss: 4.8685 - val_tf_lwlrap: 0.4133
Epoch 3/3
1171/1171 [==============================] - 15s 13ms/step - loss: 2.8375 - tf_lwlrap: 0.6390 - val_loss: 4.7888 - val_tf_lwlrap: 0.4199
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [=============

In [13]:
with open('train_arr.pickle', 'rb') as f:
    X = pickle.load(f)
    y = pickle.load(f)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

batch_size = 64
epochs = 50

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val))

Train on 7501 samples, validate on 1876 samples
Epoch 1/50
7501/7501 [==============================] - 94s 13ms/step - loss: 4.2777 - tf_lwlrap: 0.3101 - val_loss: 3.9652 - val_tf_lwlrap: 0.3627
Epoch 2/50
7501/7501 [==============================] - 95s 13ms/step - loss: 3.7359 - tf_lwlrap: 0.3962 - val_loss: 3.7580 - val_tf_lwlrap: 0.4045
Epoch 3/50
7501/7501 [==============================] - 95s 13ms/step - loss: 3.5186 - tf_lwlrap: 0.4399 - val_loss: 3.7157 - val_tf_lwlrap: 0.4067
Epoch 4/50
7501/7501 [==============================] - 95s 13ms/step - loss: 3.3439 - tf_lwlrap: 0.4724 - val_loss: 3.5190 - val_tf_lwlrap: 0.4449
Epoch 5/50
7501/7501 [==============================] - 95s 13ms/step - loss: 3.2129 - tf_lwlrap: 0.4969 - val_loss: 3.4212 - val_tf_lwlrap: 0.4683
Epoch 6/50
7501/7501 [==============================] - 95s 13ms/step - loss: 3.0951 - tf_lwlrap: 0.5268 - val_loss: 3.3521 - val_tf_lwlrap: 0.4823
Epoch 7/50
7501/7501 [==============================] - 95s 13ms

In [14]:
file_name = train_noisy['fname'].values

div = 500
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))

y_proc_tmp = one_hot(train_noisy['labels'], src_dict)

for e in range(epochs):
    k_list = random.sample(range(num_batch+1), num_batch+1)
    for k in k_list:
        with open('out/train_noisy_arr_{}.pickle'.format(k), 'rb') as f:
            X = pickle.load(f)
            y = pickle.load(f)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
        model.fit(X_train, y_train, batch_size=batch_size,epochs=3,validation_data=(X_val, y_val))
        
print('noisy iter epoch No.{} is done.'.format(e))

Train on 1172 samples, validate on 293 samples
Epoch 1/3
1172/1172 [==============================] - 15s 13ms/step - loss: 8.0338 - tf_lwlrap: 0.2785 - val_loss: 6.2932 - val_tf_lwlrap: 0.2533
Epoch 2/3
1172/1172 [==============================] - 15s 13ms/step - loss: 4.7646 - tf_lwlrap: 0.3400 - val_loss: 5.1837 - val_tf_lwlrap: 0.2356
Epoch 3/3
1172/1172 [==============================] - 15s 13ms/step - loss: 4.0578 - tf_lwlrap: 0.3891 - val_loss: 4.9858 - val_tf_lwlrap: 0.2551
Train on 1174 samples, validate on 294 samples
Epoch 1/3
1174/1174 [==============================] - 15s 13ms/step - loss: 4.6625 - tf_lwlrap: 0.2997 - val_loss: 5.2498 - val_tf_lwlrap: 0.2685
Epoch 2/3
1174/1174 [==============================] - 15s 13ms/step - loss: 4.3274 - tf_lwlrap: 0.3247 - val_loss: 4.9907 - val_tf_lwlrap: 0.2892
Epoch 3/3
1174/1174 [==============================] - 15s 13ms/step - loss: 4.0647 - tf_lwlrap: 0.3743 - val_loss: 4.8973 - val_tf_lwlrap: 0.2992
Train on 1178 samples, v

1176/1176 [==============================] - 15s 13ms/step - loss: 3.2488 - tf_lwlrap: 0.5104 - val_loss: 3.7518 - val_tf_lwlrap: 0.4558
Train on 1175 samples, validate on 294 samples
Epoch 1/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.7843 - tf_lwlrap: 0.4377 - val_loss: 4.1997 - val_tf_lwlrap: 0.4222
Epoch 2/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.5681 - tf_lwlrap: 0.4701 - val_loss: 4.1544 - val_tf_lwlrap: 0.4260
Epoch 3/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.4299 - tf_lwlrap: 0.5023 - val_loss: 4.1703 - val_tf_lwlrap: 0.4391
Train on 1169 samples, validate on 293 samples
Epoch 1/3
1169/1169 [==============================] - 15s 13ms/step - loss: 3.7461 - tf_lwlrap: 0.4317 - val_loss: 4.0646 - val_tf_lwlrap: 0.3579
Epoch 2/3
1169/1169 [==============================] - 15s 13ms/step - loss: 3.4975 - tf_lwlrap: 0.4654 - val_loss: 4.0519 - val_tf_lwlrap: 0.3516
Epoch 3/3
1169/1169 [=============

1176/1176 [==============================] - 15s 13ms/step - loss: 3.5125 - tf_lwlrap: 0.4884 - val_loss: 4.1333 - val_tf_lwlrap: 0.3935
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.3528 - tf_lwlrap: 0.5297 - val_loss: 4.1711 - val_tf_lwlrap: 0.3946
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.6426 - tf_lwlrap: 0.4610 - val_loss: 3.9664 - val_tf_lwlrap: 0.4301
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.4195 - tf_lwlrap: 0.5160 - val_loss: 4.0423 - val_tf_lwlrap: 0.4374
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.2275 - tf_lwlrap: 0.5575 - val_loss: 3.9321 - val_tf_lwlrap: 0.4487
Train on 1168 samples, validate on 292 samples
Epoch 1/3
1168/1168 [==============================] - 15s 13ms/step - loss: 3.5055 - tf_lwlrap: 0.4992 - val_loss: 3.9801 - val_tf_lwlrap: 0.4676
Epoch 2/3
1168/1168 [=============

1176/1176 [==============================] - 15s 13ms/step - loss: 3.4527 - tf_lwlrap: 0.5281 - val_loss: 3.9954 - val_tf_lwlrap: 0.4134
Epoch 2/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.2150 - tf_lwlrap: 0.5808 - val_loss: 3.8692 - val_tf_lwlrap: 0.4394
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.0140 - tf_lwlrap: 0.6128 - val_loss: 3.9056 - val_tf_lwlrap: 0.4278
Train on 1170 samples, validate on 293 samples
Epoch 1/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.5055 - tf_lwlrap: 0.4887 - val_loss: 4.1770 - val_tf_lwlrap: 0.4027
Epoch 2/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.2198 - tf_lwlrap: 0.5493 - val_loss: 4.0271 - val_tf_lwlrap: 0.4159
Epoch 3/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.0279 - tf_lwlrap: 0.5879 - val_loss: 4.0725 - val_tf_lwlrap: 0.4181
Train on 1180 samples, validate on 295 samples
Epoch 1/3
1180/1180 [=============

Train on 1180 samples, validate on 295 samples
Epoch 1/3
1180/1180 [==============================] - 15s 13ms/step - loss: 3.4706 - tf_lwlrap: 0.5089 - val_loss: 4.4025 - val_tf_lwlrap: 0.3731
Epoch 2/3
1180/1180 [==============================] - 15s 13ms/step - loss: 3.1526 - tf_lwlrap: 0.5512 - val_loss: 4.2756 - val_tf_lwlrap: 0.3884
Epoch 3/3
1180/1180 [==============================] - 15s 13ms/step - loss: 2.9908 - tf_lwlrap: 0.5921 - val_loss: 4.2527 - val_tf_lwlrap: 0.3941
Train on 1178 samples, validate on 295 samples
Epoch 1/3
1178/1178 [==============================] - 15s 13ms/step - loss: 3.4037 - tf_lwlrap: 0.4921 - val_loss: 3.9856 - val_tf_lwlrap: 0.4190
Epoch 2/3
1178/1178 [==============================] - 15s 13ms/step - loss: 3.1642 - tf_lwlrap: 0.5336 - val_loss: 3.8520 - val_tf_lwlrap: 0.4311
Epoch 3/3
1178/1178 [==============================] - 15s 13ms/step - loss: 2.9670 - tf_lwlrap: 0.5786 - val_loss: 3.8138 - val_tf_lwlrap: 0.4560
Train on 1177 samples, v

...

1176/1176 [==============================] - 15s 13ms/step - loss: 2.7027 - tf_lwlrap: 0.5947 - val_loss: 4.1175 - val_tf_lwlrap: 0.3727
Epoch 3/3
1176/1176 [==============================] - 15s 13ms/step - loss: 2.5065 - tf_lwlrap: 0.6375 - val_loss: 4.1381 - val_tf_lwlrap: 0.3753
Train on 1170 samples, validate on 293 samples
Epoch 1/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.1690 - tf_lwlrap: 0.5529 - val_loss: 4.2906 - val_tf_lwlrap: 0.4118
Epoch 2/3
1170/1170 [==============================] - 15s 13ms/step - loss: 2.8807 - tf_lwlrap: 0.6036 - val_loss: 4.1907 - val_tf_lwlrap: 0.4138
Epoch 3/3
1170/1170 [==============================] - 15s 13ms/step - loss: 2.6802 - tf_lwlrap: 0.6520 - val_loss: 4.2750 - val_tf_lwlrap: 0.4010
Train on 1176 samples, validate on 294 samples
Epoch 1/3
1176/1176 [==============================] - 15s 13ms/step - loss: 3.2043 - tf_lwlrap: 0.5421 - val_loss: 4.1897 - val_tf_lwlrap: 0.4207
Epoch 2/3
1176/1176 [=============

1175/1175 [==============================] - 15s 13ms/step - loss: 3.1654 - tf_lwlrap: 0.5792 - val_loss: 4.2982 - val_tf_lwlrap: 0.4021
Epoch 2/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.9105 - tf_lwlrap: 0.6220 - val_loss: 4.0814 - val_tf_lwlrap: 0.4431
Epoch 3/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.6998 - tf_lwlrap: 0.6585 - val_loss: 4.1348 - val_tf_lwlrap: 0.4448
Train on 1172 samples, validate on 293 samples
Epoch 1/3
1172/1172 [==============================] - 15s 13ms/step - loss: 3.0970 - tf_lwlrap: 0.5719 - val_loss: 4.1551 - val_tf_lwlrap: 0.4286
Epoch 2/3
1172/1172 [==============================] - 15s 13ms/step - loss: 2.8108 - tf_lwlrap: 0.6165 - val_loss: 4.1450 - val_tf_lwlrap: 0.4209
Epoch 3/3
1172/1172 [==============================] - 15s 13ms/step - loss: 2.6026 - tf_lwlrap: 0.6741 - val_loss: 4.2414 - val_tf_lwlrap: 0.4107
Train on 1178 samples, validate on 295 samples
Epoch 1/3
1178/1178 [=============

Train on 1175 samples, validate on 294 samples
Epoch 1/3
1175/1175 [==============================] - 15s 13ms/step - loss: 3.0772 - tf_lwlrap: 0.5993 - val_loss: 4.1736 - val_tf_lwlrap: 0.4273
Epoch 2/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.7484 - tf_lwlrap: 0.6550 - val_loss: 4.0890 - val_tf_lwlrap: 0.4494
Epoch 3/3
1175/1175 [==============================] - 15s 13ms/step - loss: 2.5689 - tf_lwlrap: 0.6935 - val_loss: 4.0854 - val_tf_lwlrap: 0.4573
Train on 1183 samples, validate on 296 samples
Epoch 1/3
1183/1183 [==============================] - 15s 13ms/step - loss: 3.1940 - tf_lwlrap: 0.5382 - val_loss: 4.5523 - val_tf_lwlrap: 0.3541
Epoch 2/3
1183/1183 [==============================] - 15s 13ms/step - loss: 2.9190 - tf_lwlrap: 0.5871 - val_loss: 4.4818 - val_tf_lwlrap: 0.3575
Epoch 3/3
1183/1183 [==============================] - 15s 13ms/step - loss: 2.6998 - tf_lwlrap: 0.6309 - val_loss: 4.4737 - val_tf_lwlrap: 0.3581
Train on 1175 samples, v

1172/1172 [==============================] - 15s 13ms/step - loss: 2.6925 - tf_lwlrap: 0.6016 - val_loss: 4.2069 - val_tf_lwlrap: 0.3911
Train on 1174 samples, validate on 294 samples
Epoch 1/3
1174/1174 [==============================] - 15s 13ms/step - loss: 3.1753 - tf_lwlrap: 0.5224 - val_loss: 3.7225 - val_tf_lwlrap: 0.3998
Epoch 2/3
1174/1174 [==============================] - 15s 13ms/step - loss: 2.8052 - tf_lwlrap: 0.5850 - val_loss: 3.6736 - val_tf_lwlrap: 0.4140
Epoch 3/3
1174/1174 [==============================] - 15s 13ms/step - loss: 2.6576 - tf_lwlrap: 0.6273 - val_loss: 3.6810 - val_tf_lwlrap: 0.4272
Train on 1170 samples, validate on 293 samples
Epoch 1/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.2898 - tf_lwlrap: 0.5176 - val_loss: 4.1253 - val_tf_lwlrap: 0.3779
Epoch 2/3
1170/1170 [==============================] - 15s 13ms/step - loss: 3.0025 - tf_lwlrap: 0.5596 - val_loss: 4.0419 - val_tf_lwlrap: 0.3855
Epoch 3/3
1170/1170 [=============

KeyboardInterrupt: 

In [14]:
with open('20190509model_curated_noisy.pickle', 'wb') as f:
    pickle.dump(model, f)

In [15]:
model.save('20190509model.h5', include_optimizer=False)

In [ ]:
with open('20190507model_curated_noisy.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
X_test_list = []

filename = glob.glob(TEST + "*")

for file in filename:
    wavfile = file
    y_proc, sr = librosa.load(wavfile)
    S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
    log_S = librosa.power_to_db(S, ref=np.max)
    X_proc = (log_S + 80) / 40 - 1
    
    num_div = X_proc.shape[1] // len_div
    num_pad = len_div - X_proc.shape[1] % len_div
    redidual_amp = np.zeros([num_freq, num_pad])
    dum = np.hstack([X_proc, redidual_amp])
    X_test_list.append(np.array(np.split(dum, num_div+1,1)))
    

In [ ]:
with open('out/test_arr.pickle', 'wb') as f:
    pickle.dump(X_test_list, f)
    pickle.dump(filename, f)

In [ ]:
with open('out/test_arr.pickle', 'rb') as f:
    X_test_list = pickle.load(f)
    filename = pickle.load(f)

In [ ]:
pred_list = []
for X_test in X_test_list:
    pred = model.predict(X_test.reshape([-1, num_freq, len_div,1])).sum(axis=0) / len(X_test)
    pred_list.append(pred)
y_pred = np.array(pred_list)

In [ ]:
sound_names = sample.columns[1:]

In [ ]:
names = []
for f in filename:
    names.append(f.split('/')[-1])

In [ ]:
se_file = pd.Series(names, name='fname')
label = pd.DataFrame(y_pred, columns=sound_names)

In [ ]:
sub_df = pd.concat([se_file, label], axis=1)

In [ ]:
sub_df.to_csv('submission.csv', index=False)